In [1]:
import pandas
import matplotlib.pyplot as plt
dataset = pandas.read_csv('Rain_data_manipulated.csv', engine='python')
dataset.head()

,Unnamed: 0,RainToday,RainTomorrow,Pressure,Humidity,Temp,WindVelocity,Brightness
0,0,0.0,0,1007.40,46.5,19.35,66.0,1.702504
1,1,0.0,0,1009.20,34.5,20.75,57.0,1.702504
2,2,0.0,0,1008.15,34.0,22.10,68.5,1.702504
3,3,0.0,0,1015.20,30.5,22.30,34.0,1.702504
4,4,0.0,0,1008.40,57.5,23.75,54.5,1.016647


In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import numpy as np

Using TensorFlow backend.


In [3]:
Labels=np.array(dataset['RainTomorrow'])

In [4]:
Samples=np.array(dataset[['RainToday','Pressure','Humidity','Temp','WindVelocity','Brightness']])

In [5]:
#*************************Do normalization on samples. Only Non-discrete columns.*************************************
Samples_normalized=[]
for j in range(6):
    arr=Samples[:,j]
    maxelement=np.amax(arr)
    minelement=np.amin(arr)
    mean=np.mean(arr)
    for i in range(len(arr)):
        arr[i]=abs(arr[i]-mean)/(maxelement-minelement)
    if j==0:
        Samples_normalized=arr
    else:
        Samples_normalized=np.concatenate((Samples_normalized,arr))

C:\Users\Om\Anaconda3\envs\opencv\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


In [12]:
Samples_normalized.shape

(142193, 1, 6)

In [6]:
Samples_normalized=Samples_normalized.reshape(Samples.shape[0],1,Samples.shape[1])

In [7]:
import numpy as np
from keras.layers import LSTM
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Flatten, Dropout, Activation, Lambda, Permute, Reshape
import json
import scipy
from keras.callbacks import Callback, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
from sklearn.metrics import confusion_matrix
import tensorflow as tf

In [8]:
def weather_predictor(X_train,Y_train,hparam):
    
    model = Sequential()

    input_shape=(X_train.shape[1],X_train.shape[2])

    model.add(LSTM(units=30,return_sequences=True,input_shape=input_shape))
    model.add(Dropout(0.2))

    model.add(LSTM(units=30))
    model.add(Dropout(0.2))
    model.add(Dense(units=20))
    model.add(Dense(units=10))
    model.add(Dense(units=4))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='adam', loss="binary_crossentropy",metrics=['accuracy'])
    trainer=model.fit(X_train, Y_train, epochs=hparam.epoch,batch_size=hparam.batch,validation_split=hparam.eval_split)
    
    plt.plot(trainer.history['accuracy'])
    plt.plot(trainer.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'val'], loc='upper left')
    plt.show()
    plt.plot(trainer.history['loss'])
    plt.plot(trainer.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'val'], loc='upper left')
    plt.show()

    return model

In [13]:
X_train,X_test,Y_train,Y_test=train_test_split(Samples_normalized,Labels,test_size=0.2,random_state=42)

In [14]:
class hparam_config():
      def __init__(self, epoch,batch,eval_split):
        self.epoch=epoch
        self.batch=batch
        self.eval_split=eval_split

param_list=hparam_config(epoch=10,batch=100,eval_split=0.2)

In [ ]:
wraps=weather_predictor(X_train,Y_train,param_list)

Train on 91003 samples, validate on 22751 samples
Epoch 1/10
91003/91003 [==============================] - 18s 202us/step - loss: nan - accuracy: 3.1867e-04 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/10
30000/91003 [========>.....................] - ETA: 8s - loss: nan - accuracy: 0.0000e+00

In [ ]:
Y_predproba=wraps.predict(X_test,Y_test)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [ ]:
lr_auc = roc_auc_score(y_test_final,Y_predproba)
print(lr_auc)
lr_fpr, lr_tpr, _ = roc_curve(Y_test, Y_predproba)
plt.plot(lr_fpr, lr_tpr, marker='.', label='Wraps')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
Y_pred=Y_predproba>0.5
accuracy_score(Y_test,Y_pred)

In [ ]:
import scikitplot as skplt
skplt.metrics.plot_confusion_matrix(
    Y_test, 
    Y_pred,
    figsize=(12,12))

In [ ]:
from keras.models import save_model
wraps.save('Wraps_LSTM_predictor.h5')  #*****************Run this cell only for improved models.